In [6]:
import discogs_client as DiscogsClient
from rapidfuzz import process as r_process, fuzz as r_fuzz
from rich import print as rprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from music_db_finder.fuzz_helpers import obj_ratio, get_discogs_name_as_str

import dotenv
env_file_name = '../../../.env'
dotenv.load_dotenv(env_file_name, override=True)

SPOTIPY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
DISCOGS_USER_TOKEN = os.getenv('DISCOGS_USER_TOKEN')
YTMUSIC_REFRESH_TOKEN = os.getenv('YTMUSIC_REFRESH_TOKEN')
YTMUSIC_ACCESS_TOKEN = os.getenv('YTMUSIC_ACCESS_TOKEN')

discogs_client = DiscogsClient.Client('ExampleApplication/0.1', user_token=DISCOGS_USER_TOKEN)
spotify_auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
spotify_client = spotipy.Spotify(client_credentials_manager=spotify_auth_manager)

In [7]:
def match_exact_discogs(search_string, results):
    candidate = None
    for result in results:
        # res_lower = getattr(result, 'name', getattr(getattr(result, 'master', getattr(result, 'main_release', None)), 'title', 'none'))
        res_lower = get_discogs_name_as_str(result)
        # if not res_lower:
        #     continue
        names = [res_lower.lower(), res_lower.lower().rsplit(maxsplit=1)[0]]
        if search_string.lower() in names:
            if not candidate:
                candidate = result
            else:
                return None
    return candidate

def match_fuzzy_discogs(search_string, results, scorer=obj_ratio):
    kwargs = {'scorer':scorer, 'limit':None, 'score_cutoff':70}
    fuzzy_results = r_process.extract(search_string, results, **kwargs)
    return map(lambda x: x[0], fuzzy_results)

def find_band_exact(band_name):
    """Returns the artist object for searches that have only one exact match, 
    ignoring case. If multiple results are returned, returns None.
    
    It works by iterating through ALL the results and comparing search string to name
    and truncated name (one split from the right, for (1) duplicated names). If it
    finds two of the same string it exists with None.
    """
    # DONE optimization: iterate over results until two are found
    # if only 1 exists, it goes through it all, as it does now
    # if 2+ exist, it goes until it founds second
    # TODO optimization: use as chained, returning status and result
    # if only 1 exist return true and the object as a list
    # if multiple exists return false and all candidates as a list
    ### Those are contradicting functionalities, enable a switch?
    # Not required - the first one if generally faster:
    # If you want to get just the artist and it exist - you need to iterate all.
    # If you want to get just the artist and multiple exist:
    #   you have to iterate again through the next function, through all results
    #   or you give up
    # For first to be slower it needs to iterate through the list twice:
    #   that is, the second same artist must be at the very end
    #   and we can rely on search optimization that they push similar artists ahead
    # DONE: check if you've iterated through them all - for _ in result iterates all
    band_results = discogs_client.search(band_name, type='artist')
    candidate = None
    for result in band_results:
        names = [(res_lower := result.name.lower()), res_lower.rsplit(maxsplit=1)[0]]
        if band_name.lower() in names:
            if not candidate:
                candidate = result
            else:
                return None
    return candidate

def find_exact(search_string, search_type):
    results = discogs_client.search(search_string, type=search_type)
    return match_exact_discogs(search_string, results)

def find_band_fuzzy(band_name, scorer=obj_ratio):
    band_results = discogs_client.search(band_name, type='artist')
    return match_fuzzy_discogs(band_name, band_results, scorer=scorer)

def find_fuzzy(search_string, search_type, scorer=obj_ratio):
    results = discogs_client.search(search_string, type=search_type)
    return match_fuzzy_discogs(search_string, results, scorer=scorer)

def find_band_by_album(band_name, album_name, scorer=obj_ratio):
    bands = find_fuzzy(band_name, 'artist', scorer)
    for band in bands:
        print(band.name)
        print(band.releases)
        # result = find_exact(album_name, 'master')


In [36]:
band_name = 'grendel'
album_name = 'inhumane amuseme'

# band_name = 'x-rx'
# album_name = 'crank up'

# 1 - find exact
result_band = find_exact(band_name, 'artist')
print(result_band)
if result_band:
    # 2 - find album exact
    releases = result_band.releases
    result_album = match_exact_discogs(album_name, releases)
    if result_album:
        print('found exact both')
        print(result_band, result_album)
    else:
        # 3 - if not exact, find fuzzy
        candidates_album = match_fuzzy_discogs(album_name, releases)
        print('found exact band, fuzzy albumns')
        print(result_band)
        for album in candidates_album:
            print('\t', album)
# 4 - if not find fuzzy 
else:
    results_band = find_fuzzy(band_name, 'artist')
    # 5 - iterate through result and break on first exact
    for band in results_band:
        releases = band.releases
        # TODO: grab a whole list and then do both check exact and fuzzy
        # if exact fails, mark fuzzy
        # if exact passes, break
        print(len(releases))
        # result_album = match_exact_discogs(album_name, releases)
        # if result_album:
        #     print('band fuzzy album exact')
        #     print(band, result_album)
        #     break



#     # 3 - break when you find the first one
#     # 3 - in narrowed list find releases fuzzy
#     candidates = []
#     for band in results:
#         print(band)
#         releases = band.releases
#         releases_results = list(match_fuzzy_discogs(album_name, releases))
#         print(releases_results)
#         if releases_results:
#             candidates.append(releases_results)

# print(candidates)



None
257
6
2
6
3
17
1


KeyboardInterrupt: 

In [12]:
results = discogs_client.search(band, type='artist')
matched_results = match_fuzzy_discogs(band, results)
for res in matched_results:
    print(res)



<Artist 794847 'X-RX'>
<Artist 2606715 'xRxAxNxGx'>
<Artist 5285986 'Jan Teutloff'>
<Artist 101741 'Robert X. Patriot'>
<Artist 1040105 'Creature Feature'>
<Artist 4783595 'Cube-[x]'>
<Artist 7219838 'xPxUxSxSxYxOxRxGxAxSxMx'>
<Artist 7871959 'xDRUGSxRxBADx'>
<Artist 11712938 'xHxRxCx'>
<Artist 11680037 'xSxSxOxRxEx'>
<Artist 6550646 'John R. Spykes'>
<Artist 12066190 'Ax4xSxUxExKxAx8xNxAxHxNx6xWxJx8xVxSxCxMxQxRxDx3xYxUxDx'>
<Artist 4502077 'Ganksta Black'>
<Artist 3472722 'Synapsyche'>


In [39]:
som = list(find_band_fuzzy('sisters of mercy'))[1]
releases = som.releases
print(len(releases))

floodlands = discogs_client.search('floodland', type='master')
print(len(list(floodlands)))

1153
32


In [35]:
artist = discogs_client.search('combichrist', type='artist')[0]
master = discogs_client.search('Today We Are All Demons', type='master')[0]
release = discogs_client.search('Today We Are All Demons', type='release')[0]
from music_db_finder.fuzz_helpers import get_discogs_name

print(get_discogs_name(artist))
print(get_discogs_name(master))
print(get_discogs_name(release))


Combichrist
Today We Are All Demons
Today We Are All Demons


In [ ]:
print(find_band_exact('combichrist'))  # <Artist 139618 'Combichrist'>
print(find_band_exact('grendel'))  # None

print(len(list(find_band_fuzzy('combichrist'))))  # 1
print(len(list(find_band_fuzzy('grendel'))))  # 14

print(find_exact('combichrist', 'artist'))  # <Artist 139618 'Combichrist'>
print(find_exact('Today We Are All Demons', 'master'))  # <Artist 139618 'Combichrist'>
print(find_exact('Today We Are All Demons', 'release'))  # None
print(find_exact('floodland', 'master'))  # None

print(len(list(find_fuzzy('combichrist', 'artist'))))  # 1
print(len(list(find_fuzzy('Today We Are All Demons', 'master'))))  # 1
print(len(list(find_fuzzy('Today We Are All Demons', 'release'))))  # 12
print(len(list(find_fuzzy('Floodland', 'master'))))  # 2



In [13]:
def match_exact_discog(search_string, results):
    candidate = None
    for result in results:
        print(result)
        res_lower = getattr(result, 'name', getattr(getattr(result, 'master', getattr(result, 'main_release', 'none')), 'title', 'none'))
        if not res_lower:
            continue
        names = [res_lower.lower(), res_lower.lower().rsplit(' (', maxsplit=1)[0]]
        if search_string.lower() in names:
            print(names)
            if not candidate:
                candidate = result
            else:
                return None
    return candidate

search_string = 'floodland'
search_type = 'master'
results = discogs_client.search(search_string, type=search_type)
print(results[0])
print(len(results))

# import cProfile
# cProfile.run('match_exact_discog(search_string, results)')

# search_string = 'floodland'
# # release
# search_type = 'release'  # discogs_client.models.Release  # 313899   # 2836
# results = discogs_client.search(search_string, type=search_type)

# one = results[0]
# # print(one.master.title)
# print(getattr(one, 'name', getattr(getattr(one, 'master', getattr(one, 'main_release', 'none')), 'title', 'none')))

# # master
# search_type = 'master'  # discogs_client.models.Master  # 2836
# results = discogs_client.search(search_string, type=search_type)
# one = results[0]
# # print(one.main_release.title)
# print(getattr(one, 'name', getattr(getattr(one, 'master', getattr(one, 'main_release', 'none')), 'title', 'none')))


# search_string = 'the sisters of mercy'
# # release
# search_type = 'artist'
# results = discogs_client.search(search_string, type=search_type)

# one = results[0]
# # print(one.name)
# print(getattr(one, 'name', getattr(getattr(one, 'master', getattr(one, 'main_release', 'none')), 'title', 'none')))



<Master 2836 'The Sisters Of Mercy - Floodland'>
32
<Master 2836 'The Sisters Of Mercy - Floodland'>
['floodland', 'floodland']
<Master 1098759 'Oscar Mulero - Floodland EP'>
<Master 2095744 'The Sisters Of Mercy - Floodland Demos'>
<Master 1284386 "The Sisters Of Mercy - Floodland Demo's">
<Master 217391 'Floodland - Ocean Of The Lost'>
<Master 217392 'Floodland - Decay'>
<Master 430797 'Aun (2) - Full Circle'>
<Master 1693442 'Analytica - Analytica'>
<Master 1238410 'Oscar Mulero - Pro-Files'>
<Master 1380655 'Planning For Burial - Quietly'>
<Master 1704501 'Zanias - Harmaline'>
<Master 1039749 'Oscar Mulero - In Bad Company'>
<Master 92738 'Rue East - Indoor Culture Remixes - Part 2'>
<Master 2913841 'Thee Flanders - Lockdown EP'>
<Master 483349 'Ektoise - Distortions'>
<Master 613531 'Various - Battery: A Tribute To Rammstein'>
<Master 286772 'Various - London v Madrid EP'>
<Master 1078543 'Ego Likeness - The Compass EPs'>
<Master 404719 'DSTR* - Silent World EP'>
<Master 3397648 '